# QGIS approach: A simple thematic map

## Introduction
This tutorial focuses on creating static maps using network data from Open Street Map (OSM). In this first tutorial, the emphasis is on woking with colors, layout, and map elements. The goal is to get familiarized with data acquisition for QGIS and practice color choice and layouting in maps.

**QGIS downloads:**

- [example_street_network_style](QGIS-files/example_street_network_style.qml)
- [practical1_processing_steps](QGIS-files/practical1_processing_steps.model3)

## How to follow this tutorial
These instructions will guide you through a practical in QGIS. The instructions will alternate between text blocks like this and 

<div style="background-color: #ccfbff; padding: 10px; border-left: 5px solid #41e4f0; margin-bottom: 10px;">
    
Instruction blocks like these
    
</div>

Text blocks offer guidance and background information whereas instruction blocks prompt you to do something in QGIS, such as:

<div style="background-color: #ccfbff; padding: 10px; border-left: 5px solid #41e4f0; margin-bottom: 10px;">
    
- Open QGIS.
    
</div>

As this is an advanced course, the instructions will expect that the basic functionalities of QGIS and GIS concepts (layers, attribute tables, processing tools, styling etc.) are familiar to the reader. Thus, we will try to keep point-by-point instructions to a minimum.

Blocks of texts or code excerpts to copy and paste into QGIS will occassionally be given. The excerpts 

`will be provided within a code quote like this.`

```
Or in a code block like this
```

QGIS is very good for many purposes, but there are things it is simply not built for, such as creating time-series plots. **Occasionally, these tutorials will include blocks of Python code – to run them, open up the Binder workplace of this course** (check if this makes sense)

Finally, there will be example visualizations throughout the instructions. We encourage you to experiment and create maps that you like – however, if you want to recreate the examples for reference, the layer style files to do so will be shared as .qml files in the folder *QGIS-files* in each week's practical directory. Alongside the examples maps is a block that tells which style file(s) are used. For example:

<div style="background-color: #9cfcc7; padding: 10px; border-left: 5px solid #06f973; margin-bottom: 10px;">

This example uses the layer style: **example_style.qml**
    
</div>

## Prerequisites
In this practical, we will need to download street network data from OSM. The simplest way to do that within QGIS is to use the **QuickOSM** plugin. QuickOSM ([see documentation](https://docs.3liz.org/QuickOSM/)) provides a user interface to get data from OSM using the [Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API).

<div style="background-color: #ccfbff; padding: 10px; border-left: 5px solid #41e4f0; margin-bottom: 10px;">
    
1. Navigate to *Plugins -> Manage and install plugins*...

2. Type *quickosm* into the search bar and install the plugin. 

3. The plugin is now installed under *Vector* drop-down menu.
</div>

## Retrieving and modifying street network data
Here we define our area of interest and get the corresponding street network from OpenStreetMap. We will retrieve streets for Kamppi, Helsinki. OSM data is richly tagged with [keys and values](https://wiki.openstreetmap.org/wiki/Tags) – for this acquisition, we will use the [highway](https://wiki.openstreetmap.org/wiki/Key:highway) tag.

<div style="background-color: #ccfbff; padding: 10px; border-left: 5px solid #41e4f0; margin-bottom: 10px;">

1. Open QuickOSM.

2. Look at the Key/Value table the *Quick query* tab. Paste `highway` under *Key* and leave *Value* empty.
    
3. Paste `Kamppi, Helsinki, Finland` as the area of interest in the field below the Key / Value table.
    
4. Open the *Advanced* drop-down menu and de-select *Points* and *Multipolygons*. This prevents us from downloading unnecessary layers.

5. Run the query.
</div>

A line layer titled *highway_Kamppi, Helsinki, Finland* has been added to the project. 

If your project has QGIS' default coordinate reference system in use (EPSG 84 | EPSG:4326), you will see a lot of distortion on the network. We'll return to the topic of projections next week, but for now, change the the project CRS and reproject the data:

<div style="background-color: #ccfbff; padding: 10px; border-left: 5px solid #41e4f0; margin-bottom: 10px;">

1. Open project CRS settings from the bottom right corner.
2. Search for `EPSG:3067` and set the project's coordinate reference system to ETRS89/TM35FIN (EPSG:3067). This is a good default projection to use over Finland.
    - NB! This setting only affects [how the layers are drawn](https://docs.qgis.org/3.34/en/docs/gentle_gis_introduction/coordinate_reference_systems.html#on-the-fly-projection) on the map screen – the underlying data retains its CRS.
3. **Additionally**, reproject the data itself to `EPSG:3067` using the processing tool *Reproject layer*.
</div>

Check out the attribute table! The data came with lots of unneccesary columns. Let's trim the data we don't need.

<div style="background-color: #ccfbff; padding: 10px; border-left: 5px solid #41e4f0; margin-bottom: 10px;">

1. Open the processing tool *Retain fields*.
2. Select 'highway', 'surface', 'name' and 'maxspeed' on the layer *reprojected*.
3. Run the tool.
</div>

Finally, let's enrich the road data with road length information:

<div style="background-color: #ccfbff; padding: 10px; border-left: 5px solid #41e4f0; margin-bottom: 10px;">

1. Run field calculator on the output of the previous step (*Retained fields*) from the attribute table or processing toolbox.
2. Use the expression `length(@geometry)`. This adds the length of each feature in map units – meters, in this case.
</div>

### Making maps

Let's make some maps now!

#### Styling
Instead of using the standard style definition options (*graduated* for continous data and *categorized* for categorical data), let's explore rule- and data-driven styling. 

By using *data-driven overrides*, we can use expressions to modify layer style (color, width, transparency, symbology etc.) based on the values available or derived from the attribute data. Most layer features can be defined this way: look for a small symbol besides the definitions.

<img src="Figures/week1_data-driven-example.jpg" />

<div style="background-color: #ccfbff; padding: 10px; border-left: 5px solid #41e4f0; margin-bottom: 10px;">
    
Apply a data-driven override to linewidth of the road network layer by clicking the symbol next to *Width* > Edit. 
    
Paste this expression to the field that opens:
    
```
CASE WHEN 
"highway" in ('footway', 'residential', 'tertiary') 
THEN 0.3 
WHEN "highway" in ('primary', 'secondary') 
THEN 0.5
ELSE 0.15
END
```

</div>

The expression above is a conditional: all values in column *highway* that match the conditional get the corresponding outcome. For example, the outcome for *primary* roads is linewidth of 0.5 (in millimeters).

Conditionals and data-driven overrides are particularly useful when we apply the expressions to the data; no need to create new columns, we can do classification on the fly!

Let's color the street network based on streetname endings: 'katu' = street, 'kuja' = alley, and a generic class for the rest. Since these are categories with no clear hierarchy, a quolitative color scheme fits our data the best.

Again, we'll use expression, but this time rely on rule-based styling.

<div style="background-color: #ccfbff; padding: 10px; border-left: 5px solid #41e4f0; margin-bottom: 10px;">
    
1. Select *Rule-based* as the styling type for the line layer.
2. Add two more line features. Copy the style of the first feature and paste it onto the other two (*Copy symbol* > *Paste symbol*).
3. Edit the first feature. Name it *Streets*. In filter, paste `right("name", 4)='katu'`. Use whatever color you wish: the example color is a bright red `#d40a47`.
    
    a. The filter matches the last four characters from the streetname column with katu (*street*). If there's a match, the rule is applied.
    
    b. Paste the color hex code to: *Color* > *Choose color...* > *HTML notation* field.
4. Repeat for class #2.
    
    a. Name: Alleys
    
    b. Filter: `right("name", 4)='kuja'`
    
    c. Example color: `#e78119`
    
5. And the final class
    
    a. Name: Other roads
    
    b. No filter: Select *Else: Catch-all for other features*
    
    c. Example color: `#30bab0`

Now, your style panel should look something like the picture below
</div>

<img src="Figures/week1_rule-based-example.png" />

### Layout
Next, it's time to layout the map and export it. There are a thousand ways to create a good, eye-catching maps! Below is one attempt, but you are given free-hands to experiment. 

If you need a refresher on QGIS' Map composer, [check out the official training manual](https://docs.qgis.org/3.34/en/docs/training_manual/map_composer/map_composer.html).

<div style="background-color: #9cfcc7; padding: 10px; border-left: 5px solid #06f973; margin-bottom: 10px;">

The line layer in this example uses the style: **QGIS-files/example_street_network_style.qml**
    
</div>

<img src="Figures/week1_road_map_example.png" />

This is far from a perfect map, but it has some basic cartographic elements and features that are good to keep in mind when making your map. However, it is highly encouraged to experiment with feature and background colors, map elements and their placement.

<img src="Figures/week1_road_map_example_annotated.png"  />

## Replicating the processing flow of this notebook

To get a street network file styled and ready for mapping, you may use this model file to replicate this process:



<div style="background-color: #ccfbff; padding: 10px; border-left: 5px solid #41e4f0; margin-bottom: 10px;">
    
1. Download the style file and processing model in *QGIS-files* folder
2. In the top right corner of the Processing toolbox, select *Open existing model* and navigate to *practical1_processing_steps.model3*
3. Open the model and select *example_street_network_style.qml* as the *Street network style file* parameter. Run the model.
4. A new layer called *Styled network* has been added to the project.

</div>

<img src="Figures/week1_model_sc.png"  />